In [2]:
from os import getcwd
from os.path import join, dirname
import numpy as np
import pandas as pd

PATH_REPO = dirname(getcwd())
PATH_DATA = join(PATH_REPO, 'data')

# 1. Get data

In [5]:
data = pd.read_csv(join(PATH_DATA, 'MeTooHate_pp.zip'), compression='zip')

/Users/agombert/miniforge3/envs/SPACY_TF/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
print(len(data))
data = data.drop_duplicates('text_pp').copy()
len(data)

807176


672433

In [7]:
data.loc[:, 'category'] = data.loc[:, 'category'].apply(lambda x: int(x))
data.loc[:, 'length'] = data.loc[:, 'text_pp'].apply(lambda x: len(x) if type(x) == str else 0)
len(data), data.category.sum()

(672433, 88117)

In [8]:
data = data.loc[data.loc[:, 'length'] > 30, :].copy()
len(data), data.category.sum()

(614559, 84545)

# 2. Extract test set

In [9]:
np.random.seed(123)

In [42]:
test = pd.concat((data.loc[data.loc[:, 'category'] == 0, :].sample(5000, replace=False),
                data.loc[data.loc[:, 'category'] == 1, :].sample(5000, replace=False))).sample(frac=1, 
                                                                        replace=False).reset_index(drop=True)
test = test.loc[:, ['text', 'text_pp', 'category']].copy()
test.loc[:, 'category'].sum(), len(test)

(5000, 10000)

In [43]:
test.to_csv(join(PATH_DATA, 'test.csv'), index=False)

# 3. Extract a small training set

In [16]:
_data = data.loc[~data.loc[:, 'text_pp'].isin(test.loc[:, 'text_pp']), :].copy()

train = pd.concat((_data.loc[_data.loc[:, 'category'] == 0, :].sample(50, replace=False),
                _data.loc[_data.loc[:, 'category'] == 1, :].sample(50, replace=False))).sample(frac=1, 
                                                                        replace=False).reset_index(drop=True)
train = train.loc[:, ['text', 'text_pp', 'category']].copy()
train.loc[:, 'category'].sum(), len(train)

(50, 100)

In [17]:
train.to_csv(join(PATH_DATA, 'train_.csv'), index=False)

# 4. Get augmented data

In [46]:
_data = data.loc[(~data.loc[:, 'text_pp'].isin(test.loc[:, 'text_pp'])) & 
                 (~data.loc[:, 'text_pp'].isin(train.loc[:, 'text_pp'])), :].copy()

random = pd.concat((_data.loc[_data.loc[:, 'category'] == 0, :].sample(95000, replace=False),
                _data.loc[_data.loc[:, 'category'] == 1, :].sample(25000, replace=False))).sample(frac=1, 
                                                                        replace=False).reset_index(drop=True)
random = random.loc[:, ['text', 'text_pp', 'category']].copy()
random.loc[:, 'category'].sum(), len(random)

(25000, 120000)

In [47]:
random.to_csv(join(PATH_DATA, 'random.csv'), index=False)